In [34]:
path = "/nlp/scr2/nlp/ego4d/data/raw/ego4d/v1/annotations/moments_train.json"

In [35]:
import json
import pprint as pp
import os
import av
import cv2

f = open(path)
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [42]:
def crop(img, crop_wd, crop_ht):
    ht, wd, _ = img.shape
    if crop_wd > wd or crop_ht > ht:
        padding = [
                (crop_wd - wd) // 2 if crop_wd > wd else 0,
                (crop_ht - ht) // 2 if crop_ht > ht else 0,
                (crop_wd - wd + 1) // 2 if crop_wd > wd else 0,
                (crop_ht - ht + 1) // 2 if crop_ht > ht else 0,
                ]
        img = pad(img, padding, fill=0)
        ht, wd, _ = img.shape
        if crop_wd == wd and crop_ht == ht:
            return img
    crop_top = int(round((ht - crop_ht) / 2.0))
    crop_left = int(round((wd - crop_wd) / 2.0))
    return img[crop_top:crop_top+crop_wd, crop_left:crop_left+crop_ht]


def resize(img, size):
    h, w, _= img.shape
    if (w <= h and w == size) or (h <= w and h == size):
        return img
    if w < h:
        ow = size
        oh = int(size * h / w)
        return cv2.resize(img, (ow, oh), interpolation=cv2.INTER_AREA)
    else:
        oh = size
        ow = int(size * w / h)
        return cv2.resize(img, (ow, oh), interpolation=cv2.INTER_AREA)

In [ ]:
data_logger = []
ego_path = "/iris/u/nivsiyer/ego4d/"

for video in data['videos']:
    
    video_name = "/nlp/scr2/nlp/ego4d/data/raw/ego4d/v1/full_scale/" + video['video_uid'] + '.mp4'
    try: 
        reader = av.open(video_name)
    except:
        print("Issue with opening the video path:", video_name)
        assert(False)
        
    # Convert video into a list of images
    images = []
    vidcap = cv2.VideoCapture(video_name)
    success, image = vidcap.read()
    image = resize(image, 64)
    image = crop(image, 64, 64)
    count = 0
    while success:
        images.append(image)     # save frame as JPEG file      
        success,image = vidcap.read()
        if success:
            image = resize(image, 64)
            image = crop(image, 64, 64)
        count += 1
        
    clips = video['clips']
    # print(clips)
    for clip in clips:
        annotations = clip['annotations']
        for annot in annotations:
            labels = annot['labels']
            for label in labels:
                start_time: label['video_start_time']
                end_time: label['video_end_time']
                start_frame: label['video_start_frame']
                end_frame: label['video_end_frame']
                task_name = label['label']
                
                os.makedirs(ego_path + task_name + "/" + video['video_uid'], exist_ok=True)
                
                # Save images in directory
                count = 0
                for image in images[start_frame:end_frame+1]:
                    cv2.imwrite(ego_path + label + "/%s.jpg" % (count), image) 
                    count += 1
                    
    break